In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import datetime as dt
import time
rule = '3T'
plt.style.use('ggplot')
pd.set_option('display.max_rows', None)

df = pd.read_csv('TWF_Futures_Minute_Trade.txt')
df.index = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df['Hour'] = df.index.map(lambda x: x.hour)
df.columns = ['date', 'time', 'open', 'high', 'low','close','volume','hour']
Morning = df[(df['hour'] >= 8) & (df['hour'] <= 13)]
Morning.index += dt.timedelta(minutes=15) # 往前調整15mins
Morning.resample(rule=rule, closed='right', label='right').first()[['open']].iloc[0:3]

rule = '60T'
Morning = df[(df['hour'] >= 8) & (df['hour'] <= 13)]
Morning.index = Morning.index + dt.timedelta(minutes=15)
d1 = Morning.resample(rule=rule, closed='right', label='left').first()[['open']]
d2 = Morning.resample(rule=rule, closed='right', label='left').max()[['high']]
d3 = Morning.resample(rule=rule, closed='right', label='left').min()[['low']]
d4 = Morning.resample(rule=rule, closed='right', label='left').last()[['close']]
d5 = Morning.resample(rule=rule, closed='right', label='left').sum()[['volume']]
df_Morning = pd.concat([d1,d2,d3,d4,d5], axis=1)
df_Morning = df_Morning.dropna()
df_Morning.index = df_Morning.index - dt.timedelta(minutes=15)

Night = df[(df['hour'] < 8) | (df['hour'] > 13)]
d1 = Night.resample(rule=rule, closed='right', label='left').first()[['open']]
d2 = Night.resample(rule=rule, closed='right', label='left').max()[['high']]
d3 = Night.resample(rule=rule, closed='right', label='left').min()[['low']]
d4 = Night.resample(rule=rule, closed='right', label='left').last()[['close']]
d5 = Night.resample(rule=rule, closed='right', label='left').sum()[['volume']]
df_Night = pd.concat([d1,d2,d3,d4,d5], axis=1)
df_Night = df_Night.dropna()

df_Day = pd.concat([df_Morning, df_Night], axis=0) #先日再夜
df_Day = df_Day.sort_index(ascending=True) #按照時間
df_Morning['Hour'] = df_Morning.index.map(lambda x: x.hour)
trainData = df_Morning[(df_Morning.index >= '2011-01-01 00:00:00') & (df_Morning.index <= '2018-12-31 00:00:00')].copy()
testData = df_Morning[(df_Morning.index >= '2019-1-1 00:00:00')].copy()
settlementDate_ = pd.read_csv('settlementDate.csv') #, encoding = 'ANSI'
settlementDate_.columns = ['settlementDate', 'futures', 'settlementPrice']
bool_ = [False if 'W' in i else True for i in settlementDate_['futures']] # if第三周->true, else: false
settlementDate = [pd.to_datetime(i).date() for i in list(settlementDate_[bool_]['settlementDate'])] #擷取第三個禮拜的日期資料


In [ ]:

fund = 1000000
FEE = 600
LENGTH = 20
NUMSTD = 1.53
Kup = 0.04
Kdown= 0.04
trainData['MA'] = trainData['close'].rolling(window=LENGTH, center=False).mean()
trainData['STD'] = trainData['close'].rolling(window=LENGTH, center=False).std()
trainData['upLine'] = trainData['MA'] + NUMSTD*trainData['STD'] #布林通道上軌
trainData['downLine'] = trainData['MA'] - NUMSTD*trainData['STD'] #布林通道下軌